In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

#spacy : 파이썬에서 자연어 전처리를 위한 오픈소스 라이브러리
import spacy
import numpy as np

import random
import math
import time

In [2]:
#같은 패턴의 random 함수를 생성을 위해 SEED 1234로 셋팅
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
#독일어와 영어 spacy Model Load, 한국어 지원 x
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [4]:
#text를 string of list로 tokenizes
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]
def tokenize_en(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

In [5]:
#Tensor로 변환하기 위한 지시 사항과 함께 데이터 유형 정의
#Field 객체는 토큰화 방법 및 생성해야 하는 Tensor의 종류와 같이 데이터 유형을 수치화 하는 방법과 관련된 매개 변수 보유
#lower : FIeld의 텍스트를 소문자로 할지의 여부 기본값 : False
SRC = Field(tokenize=tokenize_de, init_token= '<sos>', eos_token = '<eos>', lower = True)
TRG = Field(tokenize=tokenize_en, init_token= '<sos>', eos_token = '<eos>', lower = True)

In [6]:
#data load, Multi30k : 소형 데이터세트 WMT 2016
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

In [7]:
#데이터 세트에서 필드의 Vocab 오브젝트 구성, min_freq : 2번 이상의 어휘의 빈도가 발생 했을 때 Token에 포함
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [8]:
#gpu사용 가능하면 사용 아니면 cpu 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
#Batch size = 128
BATCH_SIZE = 128
#BucketIterator : 각각의 새로운 에코프에 대해 새로 섞인 배치를 생성
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

## Model
### Encoder
BiRNN사용, 각 계층에 두개의 layer 존재
output = [src len, batch size, hid dim * num direction]
hidden size : [n layers * num directions, batch size, hid dim]

In [10]:
class Encoder(nn.Module):
    #input_dim : input dimension
    #emb_dim : input data의 annoataion dimension
    #enc_hid_dim : encoder의 hidden state dimension
    #dec_hid_dim : decoder의 hidden state dimension
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        #n_layer = 1
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        #nn.Liear = y = xA^T
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
         
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden [-2, :, :] 는 마지막 step을 본 후, 첫번째 stack으로 hidden state를 제공
        #hidden [-1, :, :] 는 마지막 step을 본 후, 마지막 stack로 hidden state 제공\
        
        #torch.cat hidden[-2, :, :]와 hidden[-1, :, :]을 hidden의 tensor로 합쳐 주기 위한 것
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [11]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        #selt.attn -> input :enc_hid_dim *2 +dec_hid_dim, outpu : dec_hid_dim
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #tensor를 1로 채우고, [src_len, batch size, hid dim] 크기의 tensor 생성
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        #permute : 축 변경 encoder_output : [batch size, src len, enc hid dim*2]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
    
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        # $$\hat{a}_t = v E_t$$
        # v는 모든 엔코더의 숨겨진 상태에서 가중된 에너지 합의 가중치
        # v는 학습 parameter
        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        #dim : soft max가 계산되는 dimmension
        return F.softmax(attention, dim=1)

In [12]:
#attention layer와 attention, hidden state, encoder hidden state와 attention vector를 포함
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        # t 시점의 attention get
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
         #encoder_outputs = [src len, batch size, enc hid dim * 2]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        # w = Encoder의 hidden state와 attention을 이용해서 weighted sum을 구함.
        
        #BATCH size는 유지하고 matrix 연산
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [14]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [15]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5923, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5923, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,580,387 trainable parameters


In [17]:
optimizer = optim.Adam(model.parameters())


In [18]:

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [19]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [20]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [21]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [22]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 22s
	Train Loss: 4.996 | Train PPL: 147.774
	 Val. Loss: 4.904 |  Val. PPL: 134.820
Epoch: 02 | Time: 1m 28s
	Train Loss: 4.108 | Train PPL:  60.813
	 Val. Loss: 4.587 |  Val. PPL:  98.244
Epoch: 03 | Time: 1m 26s
	Train Loss: 3.483 | Train PPL:  32.573
	 Val. Loss: 3.776 |  Val. PPL:  43.656
Epoch: 04 | Time: 1m 27s
	Train Loss: 2.941 | Train PPL:  18.942
	 Val. Loss: 3.396 |  Val. PPL:  29.833
Epoch: 05 | Time: 1m 26s
	Train Loss: 2.547 | Train PPL:  12.771
	 Val. Loss: 3.274 |  Val. PPL:  26.407
Epoch: 06 | Time: 1m 27s
	Train Loss: 2.228 | Train PPL:   9.284
	 Val. Loss: 3.200 |  Val. PPL:  24.532
Epoch: 07 | Time: 1m 27s
	Train Loss: 1.970 | Train PPL:   7.174
	 Val. Loss: 3.144 |  Val. PPL:  23.186
Epoch: 08 | Time: 1m 25s
	Train Loss: 1.769 | Train PPL:   5.865
	 Val. Loss: 3.167 |  Val. PPL:  23.725
Epoch: 09 | Time: 1m 27s
	Train Loss: 1.592 | Train PPL:   4.911
	 Val. Loss: 3.260 |  Val. PPL:  26.053
Epoch: 10 | Time: 1m 25s
	Train Loss: 1.460 | Train PPL

In [23]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.197 | Test PPL:  24.447 |
